In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST
import torchvision.transforms as transforms
from tqdm import tqdm

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
fashion_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
# Загрузка Fashion MNIST
train_fashion = FashionMNIST(root='.', train=True, download=True, transform=fashion_transforms)
test_fashion = FashionMNIST(root='.', train=False, download=True, transform=fashion_transforms)

train_loader_fashion = DataLoader(train_fashion, batch_size=64, shuffle=True, num_workers=4)
test_loader_fashion = DataLoader(test_fashion, batch_size=64, shuffle=False, num_workers=4)

In [ ]:
# Сверточные слои AlexNet (замороженные)
features = nn.Sequential(
    nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2),
    nn.ReLU(inplace=True),
    nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2), 
    nn.MaxPool2d(kernel_size=3, stride=2),

    nn.Conv2d(96, 256, kernel_size=5, padding=2),
    nn.ReLU(inplace=True),
    nn.LocalResponseNorm(size=5, alpha=0.0001, beta=0.75, k=2),
    nn.MaxPool2d(kernel_size=3, stride=2),

    nn.Conv2d(256, 384, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),

    nn.Conv2d(384, 384, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),

    nn.Conv2d(384, 256, kernel_size=3, padding=1),
    nn.ReLU(inplace=True),
    nn.MaxPool2d(kernel_size=3, stride=2),
)

In [ ]:
# отключаем пересчёт весов для свёрточных слоёв (заморозка)
for param in features.parameters():
    param.requires_grad = False

In [ ]:
new_classifier = nn.Sequential( # новый классификатор под 10 классов
    nn.Dropout(p=0.5),
    nn.Linear(6*6*256, 4096),
    nn.ReLU(inplace=True),
    
    nn.Dropout(p=0.5),
    nn.Linear(4096, 4096),
    nn.ReLU(inplace=True),
    
    nn.Linear(4096, 10)
)

In [ ]:
alexnet_transfer = nn.Sequential(
    features,
    nn.Flatten(),
    new_classifier
).to(device)

In [ ]:
def check_accuracy(loader, model, device):
    model.eval()
    num_correct = 0
    num_samples = 0
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            scores = model(x)
            _, predictions = scores.max(dim=1)
            num_correct += (predictions == y).sum().item()
            num_samples += predictions.size(0)
    model.train()
    return (num_correct / num_samples) * 100

In [ ]:
def train_epoch(model, loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    loop = tqdm(loader, desc='Training', leave=True)
    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device)
        targets = targets.to(device)
        scores = model(data)
        loss = criterion(scores, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        loop.set_postfix(loss=loss.item())
    return running_loss / len(loader)

In [ ]:
# Обучение
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(alexnet_transfer.parameters(), lr=0.0001)

print("\n=== Transfer Learning: AlexNet → Fashion MNIST ===")
for epoch in range(1, 11):
    print(f"\n--- epoch {epoch}/10 ---")
    train_loss = train_epoch(alexnet_transfer, train_loader_fashion, criterion, optimizer, device)
    test_accuracy = check_accuracy(test_loader_fashion, alexnet_transfer, device)
    print(f"avg loss per epoch: {train_loss:.4f}")
    print(f"accuracy on test: {test_accuracy:.2f}%")